# Using continous and categorical only from transition matrices the category was obtained to filter data by category, but transi

In [69]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance,loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import stats
import dask_ml.cluster
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf , applyCosineDistance
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [70]:
covers = loadCoverGroups()
originalSongId = covers[0][0]
coverSongId = covers[0][2]
listCovers = covers[0]

In [71]:
spotifyFeatures, transitionMatrices, df = loadDataFrames()

In [72]:
ids = transitionMatrices.index.tolist()

In [73]:
transitionMatrices['id'] = ids

In [74]:
df = transitionMatrices

In [75]:
df

,0,1,2,3,4,5,6,7,8,9,...,1286,1287,1288,1290,1291,1292,1293,1294,1295,id
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,23
105,0.185185,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,105
118,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.044944,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,118
120,0.029412,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,120
292,0.013889,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002496,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002496
6002497,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002497
6002498,0.000000,0.015385,0.0,0.015385,0.0,0.015385,0.030769,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002498


In [76]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)

In [77]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, df)
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, df)

In [78]:
ranksDFCosine  = applyCosineDistance(df, ids, originalSongVector)
ranksDFCosine

,id,rank
0,180849,0.000000
1,2846348,0.059772
2,1186397,0.073942
3,532785,0.094375
4,2689908,0.096262
...,...,...
5975,6000111,1.000000
5976,6002202,1.000000
5977,6001653,1.000000
5978,6002003,1.000000


In [79]:
ranksDFCosine.loc[(ranksDFCosine.id.isin(listCovers))]

,id,rank
0,180849,0.000000
26,5000007,0.189261
37,5000019,0.214096
59,5000018,0.253582
61,6001027,0.256109
80,5000011,0.271916
107,5000017,0.291968
123,5000020,0.301726
128,1686718,0.307692
221,5000012,0.363001



### Using only 20% of the most similar songs in terms of chord progressions to recommend songs

In [51]:
percentage = 20
upperLimit = math.floor((len(ranksDFCosine) * percentage) / 100)

In [52]:
ids = ranksDFCosine.iloc[0:upperLimit,:].id

In [53]:
df = pd.read_csv("./data/unified/05-allDataContinousCategoricalTransition.csv")
df = df.drop(columns=['Unnamed: 0', 'index'])

In [54]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,23,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,118,0.614,0.755,0.0276,0.016200,0.0831,0.645,92.622,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097,0.029412,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,292,0.496,0.527,0.0339,0.195000,0.1380,0.283,155.932,0.013889,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5972,6002496,0.509,0.537,0.0384,0.347000,0.0805,0.311,170.030,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5973,6002497,0.725,0.583,0.0427,0.300000,0.0665,0.762,79.984,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5974,6002498,0.399,0.787,0.0499,0.019700,0.0685,0.572,117.089,0.000000,0.015385,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5975,6002499,0.376,0.435,0.0264,0.513000,0.1380,0.304,80.002,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [55]:
df = df[df['id'].isin(ids.tolist())]

In [56]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
10,588,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
15,830,0.526,0.665,0.0243,0.407000,0.1690,0.285,158.493,0.181818,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25,2535,0.543,0.912,0.0601,0.001620,0.0565,0.836,103.731,0.005682,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
26,2553,0.804,0.693,0.0915,0.008760,0.3140,0.654,95.721,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5912,6002434,0.648,0.488,0.0360,0.289000,0.4010,0.106,104.969,0.076923,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5933,6002455,0.737,0.742,0.0441,0.218000,0.3160,0.386,119.966,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5944,6002466,0.672,0.594,0.0789,0.062600,0.1520,0.158,171.854,0.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5964,6002488,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [57]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)

In [58]:
df

,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,2,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.526,0.665,0.0243,0.407000,0.1690,0.285,158.493,0.181818,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.543,0.912,0.0601,0.001620,0.0565,0.836,103.731,0.005682,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.804,0.693,0.0915,0.008760,0.3140,0.654,95.721,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190,0.648,0.488,0.0360,0.289000,0.4010,0.106,104.969,0.076923,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1191,0.737,0.742,0.0441,0.218000,0.3160,0.386,119.966,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1192,0.672,0.594,0.0789,0.062600,0.1520,0.158,171.854,0.333333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1193,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


## Pipeline

In [59]:
dfCosine, dfJaccard = getEuclideanJaccardDf(df)

In [60]:
dfCosine

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,997,998,999,acousticness,danceability,energy,liveness,speechiness,tempo,valence
0,0.185185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000458,0.582,0.744,0.0405,0.0336,133.344,0.674
1,0.101351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.316000,0.726,0.537,0.0707,0.1140,96.889,0.882
2,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.407000,0.526,0.665,0.1690,0.0243,158.493,0.285
3,0.005682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001620,0.543,0.912,0.0565,0.0601,103.731,0.836
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.008760,0.804,0.693,0.3140,0.0915,95.721,0.654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.289000,0.648,0.488,0.4010,0.0360,104.969,0.106
1191,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.218000,0.737,0.742,0.3160,0.0441,119.966,0.386
1192,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.062600,0.672,0.594,0.1520,0.0789,171.854,0.158
1193,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.390000,0.760,0.838,0.1660,0.0510,159.938,0.863


### Get Rankings Euclidean

In [21]:
dfCosine = dfCosine.copy()
dfCosine = removeConstantFeatures(dfCosine)
# dfCosine= applyBoxCoxAllFeatures(dfCosine)
# dfCosine = centerAndScale(dfCosine)

In [22]:
scaler = MinMaxScaler()
scaler.fit(dfCosine)
dfCosine = pd.DataFrame(scaler.transform(dfCosine)).copy()

### Original Song & Cover Song

In [23]:
originalSongId = 180849 # The Scientist Original
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfCosine)
coverSongId = 5000011 # The Scientist Original
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfCosine)

### Get Rankings Euclidean

In [24]:
ranksDFCosine = applyCosineDistance(dfCosine, ids, originalSongVector)
ranksDFCosine

,id,rank
0,180849,0.000000
1,1025154,0.172011
2,6001027,0.179779
3,2065851,0.193985
4,2689908,0.196397
...,...,...
1190,6000641,0.762459
1191,6000086,0.767585
1192,64805,0.778107
1193,6001224,0.778940


In [25]:
ranksDFCosine[ranksDFCosine.id == 180849]

,id,rank
0,180849,0.0


In [26]:
ranksDFCosine[ranksDFCosine['id']==coverSongId]

,id,rank
12,5000011,0.229652


In [27]:
ranksDFCosine.loc[(ranksDFCosine['id'] > 5000000) & (ranksDFCosine['id'] < 5900000)]

,id,rank
12,5000011,0.229652
16,5000017,0.237524
20,5000018,0.248658
24,5000019,0.254719
38,5000013,0.272556
56,5000001,0.289601
184,5000012,0.356521
317,5000007,0.393363
700,5000008,0.478705
998,5000020,0.555424


### Get Rankings Jaccard

In [28]:
originalSongId = 180849 # The Scientist Original
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfJaccard)
coverSongId = 5000011 # The Scientist Original
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfJaccard)

In [29]:
ranksDFJaccard  = applyJaccard(dfJaccard, ids, originalSongVector)

In [30]:
ranksDFJaccard

,id,rank
0,6001258,0.0
1,6001027,0.0
2,180849,0.0
3,1029726,0.0
4,466649,0.0
...,...,...
1190,780945,1.0
1191,388558,1.0
1192,821402,1.0
1193,136060,1.0


In [31]:
ranksDFJaccard[ranksDFJaccard['id']==coverSongId]

,id,rank
649,5000011,0.75


In [32]:
ranksDFJaccard[ranksDFJaccard['id']==coverSongId]

,id,rank
649,5000011,0.75


In [33]:
ranksDFJaccard[ranksDFJaccard['id'] > 5000000]

,id,rank
0,6001258,0.000000
1,6001027,0.000000
6,6002496,0.333333
8,5000001,0.333333
9,5000017,0.333333
...,...,...
1177,6001247,0.888889
1183,6001272,0.888889
1185,6001318,0.888889
1186,6001355,0.888889


### Experiment to unify rangs

In [34]:
ranks = pd.merge(ranksDFJaccard,ranksDFCosine, on='id')

In [35]:
ranks['new_rank'] = ranks['rank_x'] + ranks['rank_y']

In [36]:
ranks = ranks.sort_values(by='new_rank')
ranks.reset_index(drop=True, inplace=True)

In [37]:
ranks

,id,rank_x,rank_y,new_rank
0,180849,0.000000,0.000000,0.000000
1,6001027,0.000000,0.179779,0.179779
2,1029726,0.000000,0.361291,0.361291
3,6001258,0.000000,0.369260,0.369260
4,466649,0.000000,0.403035,0.403035
...,...,...,...,...
1190,79223,0.750000,0.834110,1.584110
1191,357373,0.888889,0.713348,1.602237
1192,6000115,0.888889,0.745536,1.634425
1193,136060,1.000000,0.662065,1.662065


In [38]:
ranks[ranks.id > 5000000]

,id,rank_x,rank_y,new_rank
1,6001027,0.000000,0.179779,0.179779
3,6001258,0.000000,0.369260,0.369260
9,5000017,0.333333,0.237524,0.570857
11,5000019,0.333333,0.254719,0.588052
12,6000128,0.333333,0.265927,0.599260
...,...,...,...,...
1186,6001798,0.888889,0.650069,1.538958
1187,6000963,0.888889,0.658220,1.547108
1188,6001546,0.888889,0.661942,1.550831
1192,6000115,0.888889,0.745536,1.634425


In [39]:
ranks.head(10)

,id,rank_x,rank_y,new_rank
0,180849,0.000000,0.000000,0.000000
1,6001027,0.000000,0.179779,0.179779
2,1029726,0.000000,0.361291,0.361291
3,6001258,0.000000,0.369260,0.369260
4,466649,0.000000,0.403035,0.403035
5,450492,0.000000,0.478506,0.478506
6,2065851,0.333333,0.193985,0.527319
7,1186397,0.333333,0.204437,0.537771
8,3504560,0.333333,0.212848,0.546181
9,5000017,0.333333,0.237524,0.570857


In [40]:
ranks.loc[(ranks.id > 5000000) & (ranks.id < 5900000)]

,id,rank_x,rank_y,new_rank
9,5000017,0.333333,0.237524,0.570857
11,5000019,0.333333,0.254719,0.588052
16,5000001,0.333333,0.289601,0.622935
54,5000018,0.571429,0.248658,0.820086
64,5000013,0.571429,0.272556,0.843984
86,5000020,0.333333,0.555424,0.888757
192,5000011,0.750000,0.229652,0.979652
312,5000008,0.571429,0.478705,1.050134
439,5000012,0.750000,0.356521,1.106521
536,5000007,0.750000,0.393363,1.143363
